![imagen con dos cerebros enfrentados, el de la izquierda azul y el de la derecha naranja de los que salen ramificaciones](../img/img_cabecera.PNG)

# Transformación y limpieza de los datos sobre el comportamiento de personas introvertidas y extrovertidas

Para poder continuar con el análisis, es necesario realizar una serie de cambios que nos faciliten conseguir los resultados esperados.

La exploración inicial se ha realizado en el archivo de ["exploración"](exploracion.ipynb).

La estructura a seguir es la siguiente:

- [0. Importación de librerías y carga de datos](#0-importación-librerías-y-carga-de-datos)
- [1. Transformaciones](#1transformaciones)
- [2. Limpieza de valores nulos](#2-limpieza-de-valores-nulos)
    - [2.1 Variables categóricas](#21-variables-categóricas)
    - [2.2 Variables numéricas](#22-variables-numéricas)
- [3. Exportación del csv transformado](#3-exportación-del-csv-transformado)



### 0. Importación librerías y carga de datos

In [1]:
# Librerias necesarias:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")

import src.transformaciones as tr

Funciones ejecutadas correctamente


In [2]:
# Carga del csv:
df = pd.read_csv('../data/personality_dataset.csv')

### 1.Transformaciones

El primer paso de limpieza es normalizar los títulos de las columnas convirtiéndolos todos a minúsculas y eliminando posibles espacios. Esto evitará errores al referenciar columnas en futuras operaciones.

In [3]:
df.columns = df.columns.str.lower().str.strip()
print(df.columns)

Index(['time_spent_alone', 'stage_fear', 'social_event_attendance',
       'going_outside', 'drained_after_socializing', 'friends_circle_size',
       'post_frequency', 'personality'],
      dtype='object')


Se ejecuta la función *clasificador_variables* para obtener los nombres de las columnas modificadas y realizar correctamente los siguientes pasos del análisis.

In [4]:
target, numericas, categoricas = tr.clasificador_variables(df, 'personality')

print(f'Columna objetivo: {target}')
print(f'Columnas numéricas: {numericas}')
print(f'Columnas categóricas: {categoricas}')


Columna objetivo: personality
Columnas numéricas: ['time_spent_alone', 'social_event_attendance', 'going_outside', 'friends_circle_size', 'post_frequency']
Columnas categóricas: ['stage_fear', 'drained_after_socializing']


Para mejorar el uso de memoria y optimizar las visualizaciones y conteos, se procede a cambiar el tipo de las variables categoricas como *category* en vez de *object*. De este modo, como la variable objetivo también presenta valores de este tipo, se crea una variable nueva para facilitar las transformaciones y el análisis. También se cambia el tupo de las variables numéricas ya que son variables discretas pero aparecen como *floats*.

In [5]:
target_categoricas = [target] + categoricas

print(target_categoricas)

['personality', 'stage_fear', 'drained_after_socializing']


In [6]:
tr.cambiar_tipos(df,target_categoricas,numericas)

,time_spent_alone,stage_fear,social_event_attendance,going_outside,drained_after_socializing,friends_circle_size,post_frequency,personality
0,4,No,4,6,No,13,5,Extrovert
1,9,Yes,0,0,Yes,0,3,Introvert
2,9,Yes,1,2,Yes,5,2,Introvert
3,0,No,6,7,No,14,8,Extrovert
4,3,No,9,4,No,8,5,Extrovert
...,...,...,...,...,...,...,...,...
2895,3,No,7,6,No,6,6,Extrovert
2896,3,No,8,3,No,14,9,Extrovert
2897,4,Yes,1,1,Yes,4,0,Introvert
2898,11,Yes,1,<NA>,Yes,2,0,Introvert


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   time_spent_alone           2837 non-null   Int64   
 1   stage_fear                 2827 non-null   category
 2   social_event_attendance    2838 non-null   Int64   
 3   going_outside              2834 non-null   Int64   
 4   drained_after_socializing  2848 non-null   category
 5   friends_circle_size        2823 non-null   Int64   
 6   post_frequency             2835 non-null   Int64   
 7   personality                2900 non-null   category
dtypes: Int64(5), category(3)
memory usage: 136.4 KB


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   time_spent_alone           2837 non-null   float64 
 1   stage_fear                 2827 non-null   category
 2   social_event_attendance    2838 non-null   float64 
 3   going_outside              2834 non-null   float64 
 4   drained_after_socializing  2848 non-null   category
 5   friends_circle_size        2823 non-null   float64 
 6   post_frequency             2835 non-null   float64 
 7   personality                2900 non-null   category
dtypes: category(3), float64(5)
memory usage: 122.3 KB


A continuación se cuentan los valores de cada variable categórica. Esto nos permitirá ver la necesidad de normalizar los datos así como la existencia de valores únicos.

In [8]:
tr.contar_valores_categoricos(df,target_categoricas)

Columna personality:
personality
Extrovert    1491
Introvert    1409
Name: count, dtype: int64


Columna stage_fear:
stage_fear
No     1417
Yes    1410
NaN      73
Name: count, dtype: int64


Columna drained_after_socializing:
drained_after_socializing
No     1441
Yes    1407
NaN      52
Name: count, dtype: int64




En cuanto a las transformaciones de estas columnas, para normalizar los datos, también es recomendable pasarlo todo a minúscula y eliminar cualquier posible espacio que haya delante o detrás de los valores. Otro cambio que sería necesario para aplicar modelos predictivos sería pasar los valores a 0 y 1. Sin embargo, para que las visualizaciones posteriores se vean con mayor claridad omitiremos este paso.

In [10]:
df_2 = tr.normalizar_valores_categoricos(df,target_categoricas)

In [11]:
df_2.head()

,time_spent_alone,stage_fear,social_event_attendance,going_outside,drained_after_socializing,friends_circle_size,post_frequency,personality
0,4,no,4,6,no,13,5,extrovert
1,9,yes,0,0,yes,0,3,introvert
2,9,yes,1,2,yes,5,2,introvert
3,0,no,6,7,no,14,8,extrovert
4,3,no,9,4,no,8,5,extrovert


### 2. Limpieza de valores nulos

Una vez normalizados todos los datos, procedemos a ver los nulos en detalle.

Aunque a rasgos generales vemos que ninguna columna presenta más del 3% de valores nulos, se procede a realizar un examen más exhaustivo para ver su distribución dentro de cada tipo de variables.

In [12]:
df_2.isna().mean()*100

time_spent_alone             2.172414
stage_fear                   2.517241
social_event_attendance      2.137931
going_outside                2.275862
drained_after_socializing    1.793103
friends_circle_size          2.655172
post_frequency               2.241379
personality                  0.000000
dtype: float64

#### 2.1 Variables categóricas

En este apartado usaremos la variable *categoricas* definida con anterioridad porque la columna *personality* no presenta nulos y por tanto no es necesario un análisis.

El primar paso es ver su distribución, por lo que para optimizar el código usaremos la función definida *contar_nulos*.

In [37]:
tr.contar_nulos(df_2,categoricas)

Valores nulos de stage_fear = 73
Valores nulos de drained_after_socializing = 52


Para ambas categorías se decide agruparlas según la columna objetivo y obtener así la distribución perteneciente a cada grupo. A rasgos generales, las personas que tienen pánico escénico suelen ser introvertidas así como aquellas que se sienten agotadas después de socializar.

In [38]:
df_2.groupby('personality')['stage_fear'].value_counts(dropna = False)

personality  stage_fear
extrovert    no            1338
             yes            111
             NaN             42
introvert    yes           1299
             no              79
             NaN             31
Name: count, dtype: int64

In [39]:
df_2.groupby('personality')['drained_after_socializing'].value_counts(dropna = False)

personality  drained_after_socializing
extrovert    no                           1362
             yes                           111
             NaN                            18
introvert    yes                          1296
             no                             79
             NaN                            34
Name: count, dtype: int64

La siguiente cuestión que se plantea es si todas las personas que tienen pánico escénico se sienten agotadas al socializar y viceversa. Por lo que se agrupan ambas columnas en función de la personalidad.
Los valores obtenidos nos permiten confirmar que todos los estudiantes que se sienten agotados despues de socializar tienen pánico escénico y que todos aquellos que no se sienten agotados al socializar no lo tienen.

In [40]:
df_2.groupby(['drained_after_socializing','stage_fear'])['personality'].value_counts(dropna = False)

drained_after_socializing  stage_fear  personality
no                         no          extrovert      1320
                                       introvert        79
yes                        yes         introvert      1266
                                       extrovert       111
Name: count, dtype: int64

In [41]:
df_2[(df_2['stage_fear'] == 'Yes')&(df_2['drained_after_socializing'] == 'No')]

,time_spent_alone,stage_fear,social_event_attendance,going_outside,drained_after_socializing,friends_circle_size,post_frequency,personality


Antes de proceder a las transformaciones, vemos como se distribuyen los nulos dentro de la columna *drained_after_socializing* en función de los valores de la columna *stage_fear*. De este modo, se observa que la mayoría de ellos corresponden a aquellos que presentan pánico escénico.

In [42]:
stage_yes = df_2[(df_2['stage_fear'] == 'yes') & (df_2['drained_after_socializing'].isna())].shape[0]
stage_no = df_2[(df_2['stage_fear'] == 'no') & (df_2['drained_after_socializing'].isna())].shape[0]

print(f'Si "stage_fear" es "yes" hay {stage_yes} nulos de la columna "drained_after_socializing".')
print(f'Si "stage_fear" es "no" hay {stage_no} nulos de la columna "drained_after_socializing".')

Si "stage_fear" es "yes" hay 33 nulos de la columna "drained_after_socializing".
Si "stage_fear" es "no" hay 18 nulos de la columna "drained_after_socializing".


Aplicando la misma lógica, observamos que la mayoría de nulos también se encuentran en aquellos que se sienten agotados después de socializar.

In [43]:
drained_yes = df_2[(df_2['drained_after_socializing'] == 'yes') & (df_2['stage_fear'].isna())].shape[0]
drained_no = df_2[(df_2['drained_after_socializing'] == 'no') & (df_2['stage_fear'].isna())].shape[0]

print(f'Si "drained_after_socializing" es "yes" hay {drained_yes} nulos de la columna "stage_fear".')
print(f'Si "drained_after_socializing" es "no" hay {drained_no} nulos de la columna "stage_fear".')

Si "drained_after_socializing" es "yes" hay 30 nulos de la columna "stage_fear".
Si "drained_after_socializing" es "no" hay 42 nulos de la columna "stage_fear".


Por último, solo exite una fila con ambos valores nulos.

In [49]:
df[df['stage_fear'].isna() & df['drained_after_socializing'].isna()]

,time_spent_alone,stage_fear,social_event_attendance,going_outside,drained_after_socializing,friends_circle_size,post_frequency,personality
1517,4.0,NaN,3.0,0.0,NaN,2.0,0.0,introvert


El análisis observado hace referencia a que todas las personas que tinen pánico escénico se sienten agotadas después de socializar y viceversa. Además indica que las extrovertidas no suelen tener pánico escénico y tampoco se sienten agotadas al socializar mientras que aquellas que son introvertidas, en su mayoría si que tienen pánico escénico y también se sienten agotadas después de socializar.

Aunque se podría haber usado esta lógica para imputar los valores nulos de estas variables, la decisión tomada es eliminar las filas ya que no superan el 3% de los datos, lo que no afectará significativamente a los análisis estadísticos. Además, de este modo evitaremos sesgos o ruido.

#### 2.2 Variables numéricas

Al igual que con las variables categóricas es necesario observar sus valores estadísticos en función de la variable objetivo para tomar una decisión.

En primer lugar observamos como la media y la mediana coinciden en todas las variables, lo que indica que al haberlos agrupados por *personality* conseguimos valores más homogéneos. Sin embargo, sí que existe diferencia entre la moda y estos valores ya que en datasets como este, que contiene escalas discretas y ordinales, es muy común que estos valores no coinciden.

In [44]:
tr.comparativa_media_mediana_moda(df_2,numericas,target)

Media de time_spent_alone según personality:
personality
extrovert    2.0
introvert    7.0
Name: time_spent_alone, dtype: float64


Mediana de time_spent_alone según personality:
personality
extrovert    2.0
introvert    7.0
Name: time_spent_alone, dtype: float64


Moda de time_spent_alone según personality:
personality
extrovert    3.0
introvert    9.0
Name: time_spent_alone, dtype: float64

---------------------------------------------------------

Media de social_event_attendance según personality:
personality
extrovert    6.0
introvert    2.0
Name: social_event_attendance, dtype: float64


Mediana de social_event_attendance según personality:
personality
extrovert    6.0
introvert    2.0
Name: social_event_attendance, dtype: float64


Moda de social_event_attendance según personality:
personality
extrovert    4.0
introvert    2.0
Name: social_event_attendance, dtype: float64

---------------------------------------------------------

Media de going_outside según personality:
person

Como en el caso anterior, estas variables tampoCo superan el 3% de los nulos. Por lo tanto, se eliminarán todas las filas con nulos, tanto en variables numéricas como categóricas. Esto deja  un totoal de 2477 filas de las 2900 originales, lo que equivale al 85% de los datos originales que son suficientes para realizar un análisis posterior. 

In [45]:
df_clean = df_2.dropna()

In [46]:
df_clean.shape

(2477, 8)

Para ver cómo han afectado estos cambios a los valores estadísticos de las variables numéricas, usamos de nuevo la función *comparativa_media_mediana_moda*. Tras ello, observamos que los valores que han cambiado hacen referencia a las modas de los siguientes grupos: 
- Extroertidos:
    - *time_spent_alone*: disminuyó de 3 a 0, desviandose 2 horas por debajo de la media y la mediana.
    - *going_outside*: disminuyó de 5 a 4, situándose 1 unidad por debajo de la media y la mediana.
- Introvertidos:
    - *friends_circle_size*: disminuyó de 2-3 amigos a  1 amigo, situándose 2 amigos por debajo menos de la media y la mediana(3 amigos).

In [47]:
tr.comparativa_media_mediana_moda(df_clean,numericas,target)

Media de time_spent_alone según personality:
personality
extrovert    2.0
introvert    7.0
Name: time_spent_alone, dtype: float64


Mediana de time_spent_alone según personality:
personality
extrovert    2.0
introvert    7.0
Name: time_spent_alone, dtype: float64


Moda de time_spent_alone según personality:
personality
extrovert    0.0
introvert    9.0
Name: time_spent_alone, dtype: float64

---------------------------------------------------------

Media de social_event_attendance según personality:
personality
extrovert    6.0
introvert    2.0
Name: social_event_attendance, dtype: float64


Mediana de social_event_attendance según personality:
personality
extrovert    6.0
introvert    2.0
Name: social_event_attendance, dtype: float64


Moda de social_event_attendance según personality:
personality
extrovert    4.0
introvert    2.0
Name: social_event_attendance, dtype: float64

---------------------------------------------------------

Media de going_outside según personality:
person

### 3. Exportación del csv transformado

El último paso es exportados los datos con las transformaciones aplicadas y sin nulos para continual con los [análisis y las visualizaciones](analisis-visualizaciones.ipynb).

In [48]:
df_clean.to_csv("../data/data_transformed_no_nul.csv",index = False)